In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/anime.csv", encoding="utf-8")

In [4]:
df["title_len"] = df["title"].map(len)

In [ ]:
df.groupby("type")["title_len"].mean().sort_values(ascending=False)

In [23]:
rating_groups = {
    "R - 17+ (violence & profanity)": "adult",
    "PG - Children": "kids",
    "PG-13 - Teens 13 or older": "teens",
    "G - All Ages": "kids",
    "R+ - Mild Nudity": "adult"
}

df["rating_group"] = df["rating"].map(rating_groups).fillna("other")

In [ ]:
type_map = {
    "TV": "tv",
    "Movie": "movie",
    "OVA": "ova"
}
df["type"] = df["type"].map(type_map).fillna(df["type"])

In [ ]:
df["desc"] = df.apply(
    lambda row: f"{row["title"]} --> {row["type"]}, {row["start_date"]}",
    axis=1
)

In [ ]:
df.groupby("duration").agg(
    {
        "members": "mean",
        "score": "mean"
    }
)

# Задача 5 невозможна так как там не только минуты и есть часы и для
# каждого создавать формотирования сложно существует 133 уникальных таких дат
# я бы сам не прочьвыполнить но это сложно для меня

In [3]:
df["year"] = df["aired"].str.extract(r"(\d{4})").astype("Int64")

In [ ]:
df["is_modern_tv"] = df.apply(
    lambda row: True if row["type"] == "tv" and row["year"] > 2010 else False,
    axis=1
)
# apply — 10 раз медленнее.
df["is_modern_tv"] = (df["type"] == "tv") & (df["year"] > 2010)


In [ ]:
df["mean_score_by_type"] = df.groupby("type")["score"].transform("mean")

In [7]:
df["score_diff"] = df["score"] - df["mean_score_by_type"]

In [ ]:
df["mean_members_by_type"] = df.groupby("type")["members"].transform("mean").sort_values(ascending=False)
df["is_top_in_type"] = df.apply(
    lambda row: True if row["members"] > row["mean_members_by_type"] else False,
    axis=1
)
df.iloc[:, 6:]

In [9]:
df["populraity"] = df["score"] * (df["members"] / 10000)

In [ ]:
def get_category(row):
    if row["score"] > 8:
        return "high"
    elif row["score"] > 6:
        return "mid"
    else:
        return "low"

df["category"] = df.apply(get_category, axis=1)
df.iloc[700:][["score", "category"]]